### Credit to this user for base code https://www.kaggle.com/ilialar/goose-on-a-healthy-diet

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### general ideas:
stay small and therefore have more control over where the tail is (since the longer your agent becomes, the less flexible your geese is.)

move away from food as most other agents will move towards it (unless approaching starvation)

when the opponent is considerably longer, chase the head of the opponent, reducing the number of options it has when turning and increasing the chance it collides with something

In [ ]:
%%writefile submission.py

from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col
import numpy as np
import random

last_step = None






def get_nearest_cells(x,y):
    # returns all cells reachable from the current one
    result = []
    for i in (-1,+1):
        result.append(((x+i+7)%7, y))
        result.append((x, (y+i+11)%11))
    return result

def find_closest_food(table):
    # returns the first step toward the closest food item
    new_table = table.copy()    
    
    # (direction of the step, axis, code)
    possible_moves = [
        (1, 0, 1),
        (-1, 0, 2),
        (1, 1, 3),
        (-1, 1, 4)
    ]
    
    # shuffle possible options to add variability
    random.shuffle(possible_moves)    
    
    updated = False
    for roll, axis, code in possible_moves:

        shifted_table = np.roll(table, roll, axis)
        
        if (table == -2).any() and (shifted_table[table == -2] == -3).any(): # we have found some food at the first step
            return code
        else:
            mask = np.logical_and(new_table == 0,shifted_table == -3)
            if mask.sum() > 0:
                updated = True
            new_table += code * mask
        if (table == -2).any() and shifted_table[table == -2][0] > 0: # we have found some food
            return shifted_table[table == -2][0]
        
        # else - update new reachable cells
        mask = np.logical_and(new_table == 0,shifted_table > 0)
        if mask.sum() > 0:
            updated = True
        new_table += shifted_table * mask

    # if we updated anything - continue recursion
    if updated:
        return find_closest_food(new_table)
    # if not - return some step
    else:
        return table.max()

def agent(obs_dict, config_dict):
    global last_step
    
    observation = Observation(obs_dict)
    configuration = Configuration(config_dict)
    player_index = observation.index
    player_goose = observation.geese[player_index]
    player_head = player_goose[0]
    player_row, player_column = row_col(player_head, configuration.columns)


    table = np.zeros((7,11))
    # 0 - empty cells
    # -1 - obstacles
    # -4 - possible obstacles
    # -2 - food
    # -3 - head
    # 1,2,3,4 - reachable on the current step cell, number is the id of the first step direction
    
    legend = {
        1: 'SOUTH',
        2: 'NORTH',
        3: 'EAST',
        4: 'WEST'
    }
    
    # let's add food to the map
    for food in observation.food:
        x,y = row_col(food, configuration.columns)
        table[x,y] = -2 # food
        
    # let's add all cells that are forbidden
    for i in range(4):
        opp_goose = observation.geese[i]
            
        is_close_to_food = False
            
        if i != player_index:
            x,y = row_col(opp_goose[0], configuration.columns)
            possible_moves = get_nearest_cells(x,y) # head can move anywhere
            
            for x,y in possible_moves:
                if table[x,y] == -2:
                    is_close_to_food = True
            
                table[x,y] = -4 # possibly forbidden cells
        
        # usually we ignore the last tail cell but there are exceptions
        tail_change = -1
        if obs_dict['step'] % 40 == 39:
            tail_change -= 1
        
        # we assume that the goose will eat the food
        if is_close_to_food:
            tail_change += 1
        if tail_change >= 0:
            tail_change = None
            

        for n in opp_goose[:tail_change]:
            x,y = row_col(n, configuration.columns)
            table[x,y] = -1 # forbidden cells
    
    # going back is forbidden according to the new rules
    x,y = row_col(player_head, configuration.columns)
    if last_step is not None:
        if last_step == 1:
            table[(x + 6) % 7,y] = -1
        elif last_step == 2:
            table[(x + 8) % 7,y] = -1
        elif last_step == 3:
            table[x,(y + 10)%11] = -1
        elif last_step == 4:
            table[x,(y + 12)%11] = -1
        
    # add head position
    table[x,y] = -3
    
    step = None
    
    if len(player_goose) < 4:
        # the first step toward the nearest food
        step = int(find_closest_food(table))
    elif len(player_goose) == 4 and len(opp_goose)<20:
        # let the opponents build up size so they're easier to catch
        head_x, head_y = row_col(player_head, configuration.columns)
        tail_x, tail_y = row_col(player_goose[-1], configuration.columns)
        
        # if the tail is close let's move to the place of tail
        if abs(head_x - tail_x) + abs(head_y - tail_y) == 1:
            if (head_x - tail_x + 7) % 7 == 1:
                step = 2
            elif (head_x - tail_x + 7) % 7 == 6:
                step = 1
            elif (head_y - tail_y + 11) % 11 == 1:
                step = 4
            elif (head_y - tail_y + 11) % 11 == 10:
                step = 3
        # else try to turn
        elif last_step in (1, 2):
            possible_steps = []
            if table[head_x,(head_y + 12)%11] == 0:
                possible_steps.append(3)
            if table[head_x,(head_y + 10)%11] == 0:
                possible_steps.append(4)
            if len(possible_steps)>0:
                step = np.random.choice(possible_steps)
        elif last_step in (3, 4):
            possible_steps = []
            if table[(head_x + 8) % 7,head_y] == 0:
                possible_steps.append(1)
            if table[(head_x + 6) % 7,head_y] == 0:
                possible_steps.append(2)
            if len(possible_steps)>0:
                step = np.random.choice(possible_steps)
                
    elif len(opp_goose)>20: # go in for the kill - approach the head of the opponent and try to force it to steer into itself/ the player body
        head_opp_x, head_opp_y = row_col(opp_goose[0], configuration.columns)
        head_x,head_y = row_col(player_goose[0], configuration.columns)
        
        dx = abs(head_opp_x - head_x)
        dy = abs(head_opp_y - head_y)
        
        possible_moves = get_nearest_cells(x,y) # filter occupied cells
        random.shuffle(possible_moves)
        for x,y in possible_moves:
            if table[x][y] == 0:
                if abs(head_opp_x - x) < dx or abs(head_opp_y - y) < dy:
                    # convert move to a step
                    if (head_x - x): # change in x
                        if (head_x > x):
                            step = 3
                        else:
                            step = 4
                    elif (head_y - y):
                        if (head_y > y):
                            step = 2
                        else:
                            step = 1
        
        
           
    # if there is not available steps try to go to safe or possibly dangerous cell
    if step not in [1,2,3,4]:
        x,y = row_col(player_head, configuration.columns)
        if table[(x + 8) % 7,y] in [0,-2]:
            step = 1
        elif table[(x + 6) % 7,y] in [0,-2]:
            step = 2
        elif table[x,(y + 12)%11] in [0,-2]:
            step = 3
        elif table[x,(y + 10)%11] in [0,-2]:
            step = 4
                
    if step not in [1,2,3,4]:
        if table[(x + 8) % 7,y] == -4:
            step = 1
        elif table[(x + 6) % 7,y] == -4:
            step = 2
        elif table[x,(y + 12)%11] == -4:
            step = 3
        elif table[x,(y + 10)%11] == -4:
            step = 4                
    # else - do a random step and lose
        else:
            step = 1
    
    last_step = step
    return legend[step]

In [ ]:
from kaggle_environments import evaluate, make
env = make("hungry_geese", debug = True)

In [ ]:
env.reset()
env.run(['submission.py', 'submission.py', 'submission.py', 'submission.py'])
env.render(mode="ipython", width=800, height=700)